In [1]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [2]:
self = MolecularDynamics(number_atoms=[50],
                      epsilon=[0.1], # kcal/mol
                      sigma=[3], # A
                      atom_mass=[1], # g/mol
                      atom_charge=[0], # in elementary charge units
                      Lx=100, # A
                      Ly=100, # A
                      Lz=100, # A
                      minimization_steps=50,
                      maximum_steps=1000,
                      desired_temperature=100,
                      desired_pressure=1,
                      thermo = 250,
                      dump = 250,  
                      tau_temp = 100, # fs
                      tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps > /dev/null")

step epot maxF
0 1151.993 14972.849
5 26.274 430.176
10 -0.171 0.437
15 -0.211 0.049
20 -0.211 0.002
25 -0.211 0.007
30 -0.212 0.002
35 -0.212 0.008
40 -0.212 0.003
45 -0.213 0.008
50 -0.213 0.008
step  N     T (K)     p (atm)   V (A3)    Ep (kcal/mol) Ek (kcal/mol) dens (g/mol/A3)
0     50    1e+02     0.666     9.99e+05  -0.213        14.6          5e-05        
250   50    99.9      0.737     8.94e+05  -0.0702       14.6          5.59e-05     
500   50    1e+02     0.798     8.21e+05  -0.129        14.6          6.09e-05     
750   50    99.7      0.856     7.72e+05  -0.0386       14.6          6.47e-05     
1000  50    99.9      0.902     7.39e+05  -0.00826      14.6          6.77e-05     


256